In [1]:
import os
import json
import numpy as np
from tqdm import tqdm
from PIL import Image

%matplotlib inline

In [2]:
root = '/home/zhmiao/datasets/ecology/CCT_15/'
img_root = os.path.join(root, 'eccv_18_all_images_sm')

In [3]:
imgs_reg = os.listdir(os.path.join(root, 'eccv_18_all_images_sm/'))

In [4]:
imgs_reg[0]

'59fae6fe-23d2-11e8-a6a3-ec086b02610b.jpg'

In [5]:
js_path = '/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_train_annotations.json'

with open(js_path, 'r') as js:
    js_file = json.load(js)

In [14]:
cat = []
for ann in tqdm(js_file['annotations']):
    img_id = ann['image_id']
    file_path = os.path.join(img_root, img_id + '.jpg')
    
    if not os.path.isfile(file_path):
        print(file_path)
        
    if ann['category_id'] != 30 and ann['category_id'] != 21 and ann['category_id'] != 51:
        cat.append(ann['category_id'])

100%|██████████| 14071/14071 [00:00<00:00, 202782.60it/s]


In [15]:
unique_cat, unique_counts = np.unique(cat, return_counts=True)

In [16]:
sorted_unique_cat = sorted(list(zip(unique_cat, unique_counts)), key=lambda x:x[1], reverse=True)

In [17]:
sorted_unique_cat

[(1, 2751),
 (10, 2490),
 (9, 1501),
 (16, 1290),
 (5, 1256),
 (3, 1120),
 (8, 919),
 (6, 791),
 (33, 668),
 (11, 619),
 (99, 377),
 (7, 230),
 (34, 45)]

In [21]:
np.random.randint(0, 10)

5

In [24]:
min_split = 100
season_1 = []
season_2 = []

for i, ent in enumerate(sorted_unique_cat):
    
    lab = ent[0]
    total_num = ent[1]
    
    if i <= 9:
        
        np.random.seed(lab)
        split = np.random.randint(min_split, (total_num - min_split))
        
        season_1.append((lab, split))
        season_2.append((lab, total_num - split))
        
    else:
        
        season_2.append(ent)
    

In [25]:
season_1

[(1, 1161),
 (10, 1389),
 (9, 482),
 (16, 781),
 (5, 967),
 (3, 974),
 (8, 551),
 (6, 327),
 (33, 120),
 (11, 509)]

In [26]:
season_2

[(1, 1590),
 (10, 1101),
 (9, 1019),
 (16, 509),
 (5, 289),
 (3, 146),
 (8, 368),
 (6, 464),
 (33, 548),
 (11, 110),
 (99, 377),
 (7, 230),
 (34, 45)]

In [28]:
unique_cat

array([ 1,  3,  5,  6,  7,  8,  9, 10, 11, 16, 33, 34, 99])

In [27]:
js_file.keys()

dict_keys(['info', 'categories', 'images', 'annotations'])

In [29]:
ann

{'image_id': '59fe2386-23d2-11e8-a6a3-ec086b02610b',
 'category_id': 11,
 'bbox': [974.08, 695.04, 227.84000000000003, 148.4799999999999],
 'id': '29856'}

In [31]:
annotation_list_dict = {k:[] for k in unique_cat}

for ann in tqdm(js_file['annotations']):
    if ann['category_id'] != 30 and ann['category_id'] != 21 and ann['category_id'] != 51:
        annotation_list_dict[ann['category_id']].append(ann)

100%|██████████| 14071/14071 [00:00<00:00, 1061895.92it/s]


In [32]:
ann_sea_1 = {'annotations': []}
ann_sea_2 = {'annotations': []}

for i in range(len(unique_cat)):
    
    lab = season_2[i][0]
    
    if i <= 9:
        
        num_sea_1 = season_1[i][1]
        
        np.random.seed(lab)
        
        ann_list = annotation_list_dict[lab]
        
        np.random.shuffle(ann_list)
        
        ann_sea_1['annotations'] += ann_list[:num_sea_1]
        ann_sea_2['annotations'] += ann_list[num_sea_1:]
        
    else:
            
        ann_list = annotation_list_dict[lab]
        ann_sea_2['annotations'] += ann_list[num_sea_1:]

In [35]:
with open('/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_train_annotations_season_1.json', 'w') as json_file:
    json.dump(ann_sea_1, json_file)

with open('/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_train_annotations_season_2.json', 'w') as json_file:
    json.dump(ann_sea_2, json_file)

In [41]:
# js_path = '/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_val_annotations.json'

season_1_labels = [e[0] for e in season_1]
season_2_labels = [e[0] for e in season_2]

def eval_ann_split(js_path):
    
    ann_sea_1 = {'annotations': []}
    ann_sea_2 = {'annotations': []}
    
    with open(js_path, 'r') as js:
        js_file = json.load(js)
    
    for ann in tqdm(js_file['annotations']):
            
        if ann['category_id'] in season_1_labels:
            ann_sea_1['annotations'].append(ann)
            
        if ann['category_id'] in season_2_labels:
            ann_sea_2['annotations'].append(ann)

    return ann_sea_1, ann_sea_2

In [42]:
cis_val_sea_1, cis_val_sea_2 = eval_ann_split('/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_val_annotations.json')
cis_test_sea_1, cis_test_sea_2 = eval_ann_split('/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_test_annotations.json')

100%|██████████| 16395/16395 [00:00<00:00, 418799.45it/s]


In [43]:
with open('/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_val_annotations_season_1.json', 'w') as json_file:
    json.dump(cis_val_sea_1, json_file)
with open('/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_val_annotations_season_2.json', 'w') as json_file:
    json.dump(cis_val_sea_2, json_file)

with open('/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_test_annotations_season_1.json', 'w') as json_file:
    json.dump(cis_test_sea_1, json_file)
with open('/home/zhmiao/datasets/ecology/CCT_15/eccv_18_annotation_files/cis_test_annotations_season_2.json', 'w') as json_file:
    json.dump(cis_test_sea_2, json_file)

In [37]:
season_1_labels

[1, 10, 9, 16, 5, 3, 8, 6, 33, 11]

In [38]:
season_2_labels

[1, 10, 9, 16, 5, 3, 8, 6, 33, 11, 99, 7, 34]

In [8]:
js_file['categories']

[{'id': 6, 'name': 'bobcat'},
 {'id': 1, 'name': 'opossum'},
 {'id': 30, 'name': 'empty'},
 {'id': 9, 'name': 'coyote'},
 {'id': 3, 'name': 'raccoon'},
 {'id': 11, 'name': 'bird'},
 {'id': 8, 'name': 'dog'},
 {'id': 16, 'name': 'cat'},
 {'id': 5, 'name': 'squirrel'},
 {'id': 10, 'name': 'rabbit'},
 {'id': 7, 'name': 'skunk'},
 {'id': 99, 'name': 'rodent'},
 {'id': 21, 'name': 'badger'},
 {'id': 34, 'name': 'deer'},
 {'id': 33, 'name': 'car'},
 {'id': 51, 'name': 'fox'}]

In [9]:
class_indices = {}
label = 0
for cat in js_file['categories']:
    if cat['id'] != 30 and cat['id'] != 33:
        class_indices[cat['id']] = label
        label += 1

In [10]:
class_indices

{6: 0,
 1: 1,
 9: 2,
 3: 3,
 11: 4,
 8: 5,
 16: 6,
 5: 7,
 10: 8,
 7: 9,
 99: 10,
 21: 11,
 34: 12,
 51: 13}